In [1]:
import urllib.request
import pandas as pd
import zipfile
import requests
import plotly.graph_objects as go
import folium 
import math

from geopy.geocoders import MapQuest
from geopy.extra.rate_limiter import RateLimiter


In [2]:
url = 'https://files.digital.nhs.uk/assets/ods/current/epraccur.zip'
filehandle, _ = urllib.request.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
first_file = zip_file_object.namelist()[0]
file = zip_file_object.open(first_file)
content = file.read()
csv_file = open('gp_data.csv', 'wb')
csv_file.write(content )
csv_file.close()
header_list = ["Organisation Code", "Name", "National Grouping", "High Level Health Geography", "Address line 1", "Address line 2", "Address line 3", 
"Address line 4", "Address line 5","Postcode","Open Date","Close Date","Status Code","Organisation Sub-Type Code","Commissioner","Join Provider/Purchaser Date",
"Left Provider/Purchaser Date","Contact Telephone Number", "Null 1", "Null 2", "Null 3", "Amended Record Indicator", "Null 4", "Provider/Purchaser",
"Null 5", "Prescribing Setting", "Null 6"]
gp_practice_df = pd.read_csv('gp_data.csv', names=header_list)
gp_practice_df.fillna('', inplace=True)
gp_practice_df['Partial Address'] = gp_practice_df[['Address line 1', 'Address line 2', 'Address line 3', 'Address line 4',]].agg(', '.join, axis=1)
gp_practice_df['Full Address'] = gp_practice_df[['Partial Address', 'Address line 5',]].agg(' '.join, axis=1)
gp_practice_df['Full Address'] = gp_practice_df['Full Address'].str.title()
gp_practice_df['Name'] = gp_practice_df['Name'].str.title()

In [3]:
gp_practice_df_1 = gp_practice_df.drop(columns = {"High Level Health Geography", "Address line 1", "Address line 2", "Address line 3", "Address line 4", 
"Address line 5", "Open Date", "Close Date", "Organisation Sub-Type Code", "Commissioner", "Join Provider/Purchaser Date", "Left Provider/Purchaser Date",
"Null 1", "Null 2", "Null 3", "Amended Record Indicator", "Null 4", "Partial Address", "Provider/Purchaser", "Null 5", "Null 6"})
gp_practice_df_2 =  gp_practice_df_1[gp_practice_df_1["Status Code"] == "A"]
gp_practice_df_3 =  gp_practice_df_2[gp_practice_df_2["Prescribing Setting"] == 4]
gp_practice_df_eng = gp_practice_df_3[gp_practice_df_3["National Grouping"].str.contains("YAC|YAD|YAE|YAF|W00")==False]
gp_practice_df_eng_1 = gp_practice_df_eng.reset_index(drop = True)
gp_practice_df_eng_2 = gp_practice_df_eng_1.copy()
gp_practice_df_eng_3 = gp_practice_df_eng_2.drop( columns = {"Status Code", "Prescribing Setting"})
gp_practice_df_ldn = gp_practice_df_eng_3[gp_practice_df_eng_3["National Grouping"].str.contains("Y56")==True]
gp_practice_df_ldn['Name'] = gp_practice_df_ldn['Name'].str.replace('Gp', 'GP')
gp_practice_df_ldn['Full Address'] = gp_practice_df_ldn['Full Address'].str.replace(' ,', ' ').str.replace('  ', ' ').str.replace('Gp', 'GP').map(lambda x: x.rstrip(', '))
gp_practice_df_ldn_2  = gp_practice_df_ldn[gp_practice_df_ldn["Name"].str.contains("Babylon")==False]
gp_practice_df_ldn_3 = gp_practice_df_ldn_2.reset_index(drop = True)


<ipython-input-3-aece39228467>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gp_practice_df_ldn['Name'] = gp_practice_df_ldn['Name'].str.replace('Gp', 'GP')
<ipython-input-3-aece39228467>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gp_practice_df_ldn['Full Address'] = gp_practice_df_ldn['Full Address'].str.replace(' ,', ' ').str.replace('  ', ' ').str.replace('Gp', 'GP').map(lambda x: x.rstrip(', '))


In [4]:
csv_url = "https://files.digital.nhs.uk/40/2232E5/gp-reg-pat-prac-all.csv"
req = requests.get(csv_url)
url_content = req.content
csv_file = open('gp_pop.csv', 'wb')
csv_file.write(url_content)
csv_file.close()
gp_pop_df = pd.read_csv('gp_pop.csv')
gp_pop_df.rename(columns={'CODE': 'Organisation Code', 'NUMBER_OF_PATIENTS': 'Number of patients registered at GP practices in England'}, inplace=True)
gp_pop_df_1 = gp_pop_df.drop(columns = {'PUBLICATION', 'EXTRACT_DATE', 'TYPE', 'CCG_CODE', 'ONS_CCG_CODE', 'SEX', 'AGE', 'POSTCODE'})


In [5]:
gp_pop_ldn = gp_practice_df_ldn_3.join(gp_pop_df_1, rsuffix='Organisation Code')
gp_pop_ldn.rename(columns={'Number of patients registered at GP practices in England': 'Number of patients registered at GP practices in London'}, inplace=True)
gp_pop_ldn["Address"] = gp_pop_ldn[["Full Address", "Postcode"]].agg(', '.join, axis=1)
gp_pop_ldn_1 = gp_pop_ldn.drop(columns={'Organisation CodeOrganisation Code', 'National Grouping', 'Postcode', 'Full Address'})




In [6]:
x0 = gp_pop_ldn_1['Number of patients registered at GP practices in London']
x1 = gp_pop_df_1['Number of patients registered at GP practices in England']
count_england = gp_pop_df_1['Number of patients registered at GP practices in England'].count()
count_london = gp_pop_ldn_1['Number of patients registered at GP practices in London'].count()
fig_1 = go.Figure()
fig_1.add_trace(go.Box(x=x0, 
boxmean=True,  
boxpoints= 'all', 
jitter=0.3, 
name="London", 
marker_color ="#0072CE", 
whiskerwidth=0.5, 
marker_size=3,
line_width=2))
fig_1.add_trace(go.Box(x=x1, 
boxmean=True, 
boxpoints= 'all', 
jitter=0.3, 
name="England", 
marker_color = "#003087", 
whiskerwidth=0.5, 
marker_size=3,
line_width=2))
fig_1.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    font = dict(family = "Arial", size = 16),
    autosize=True,
    margin=dict(l=75, r=50, b=130, t=50, pad=4, autoexpand=True), hoverlabel=dict(
        font_size=12,
        font_family="Arial"
    ), xaxis=dict(title='Number of patients registered at individual GP practices', zeroline=False))
fig_1.add_annotation(dict(font=dict(family = "Arial",size=15),
                                        x=0.33,
                                        y=-0.40,
                                        showarrow=False,
                                        text="Number of GP practices in England: %s" %count_england,
                                        textangle=0,
                                        xanchor='right',
                                        xref="paper",
                                        yref="paper"))
fig_1.add_annotation(dict(font=dict(family = "Arial",size=15),
                                        x=0.323,
                                        y=-0.46,
                                        showarrow=False,
                                        text="Number of GP practices in London: %s" %count_london,
                                        textangle=0,
                                        xanchor='right',
                                        xref="paper",
                                        yref="paper"))      
                                      

           

In [7]:
file_name = 'assets/data/gp_pop_ldn_mapped.csv'
#geolocator = MapQuest(user_agent="open_analytics_1", api_key = "TYUmkRCUitJcXB8gISceJ4zNuhS1HCum")
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

#gp_pop_ldn_1["loc"] = gp_pop_ldn_1["Address"].apply(geolocator.geocode)
#gp_pop_ldn_1["Point"]= gp_pop_ldn_1["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
#gp_pop_ldn_1[['Latitude', 'Longitude', 'Altitude']] = pd.DataFrame(gp_pop_ldn_1['Point'].to_list(), index=gp_pop_ldn_1.index)
#gp_pop_ldn_1.to_csv(file_name)



In [8]:
gp_prac_pop_df = pd.read_csv(file_name)
gp_prac_pop_df_1 = gp_prac_pop_df[gp_prac_pop_df['loc'].str.contains("US")==False]
gp_prac_pop_df_1

,Unnamed: 0,Organisation Code,Name,Postcode,Full Address,Number of patients registered at GP practices in London,loc,Point,Latitude,Longitude,Altitude
1,1,E83005,Lichfield Grove Surgery,N3 2JP,"64 Lichfield Grove, Finchley, London",12069,"London, Westminster, GB, N3 2JP","(51.599558, -0.193234, 0.0)",51.599558,-0.193234,0.0
2,2,E83006,Greenfield Medical Centre,NW2 1HS,"Greenfield Medical Centre, 143-145 Cricklewood...",18746,"London, Westminster, GB, NW2 1HS","(51.561018, -0.205684, 0.0)",51.561018,-0.205684,0.0
7,7,E83011,The Everglade Medical Practice,NW9 5XT,"Grahame Park Health Ctr., The Concourse, Graha...",6272,"London, Westminster, GB, NW9 5XT","(51.601572, -0.2442, 0.0)",51.601572,-0.244200,0.0
8,8,E83012,The Old Court House Surgery,EN5 4BB,"The Old Court House, 27 Wood Street, Barnet, H...",12703,"Barnet, Hertfordshire, GB, EN5 4BB","(51.652911, -0.201594, 0.0)",51.652911,-0.201594,0.0
11,11,E83017,Longrove Surgery,EN5 2ED,"Vale Drive Primary C/Ctr, Barnet, Hertfordshire",8057,"Barnet, Hertfordshire, GB, EN5 2ED","(51.648831, -0.194702, 0.0)",51.648831,-0.194702,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1205,1205,Y05857,Rhnd,SW15 3SW,"Rhnd, West Hill, Putney, London",26315,"London, Westminster, GB, SW15 3SW","(51.452239, -0.213652, 0.0)",51.452239,-0.213652,0.0
1207,1207,Y06345,Bromleag Care Practice,BR3 3QL,"Kelsey Park Farm House, 379 Croydon Road, Beck...",26239,"Beckenham, Kent, GB, BR3 3QL","(51.40506, -0.033175, 0.0)",51.405060,-0.033175,0.0
1208,1208,Y06487,Babylon GP At Hand Birmingham,B9 4EY,"121 Glover Street, Birmingham",15737,"Birmingham, West Midlands, GB, B9 4EY","(52.477603, -1.877822, 0.0)",52.477603,-1.877822,0.0
1209,1209,Y06545,Sel Special Allocation Practice,SE6 4JH,"Novum Health Partnership, Rushey Green Group P...",11345,"London, Westminster, GB, SE6 4JH","(51.449319, -0.018019, 0.0)",51.449319,-0.018019,0.0


In [9]:
gp_prac_pop_df_1['gp_pop_quintile'] = pd.qcut(gp_prac_pop_df_1['Number of patients registered at GP practices in London'], 5, labels=False)
colordict = {0: 'green', 1: 'lightgreen', 2: 'orange', 3: 'red', 4: 'darkred'}



<ipython-input-9-45b861efd1fa>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
frame = folium.Figure(width=900, height=500)
fig_2 = folium.Map(
    location=[51.5, -0.1],
    tiles="cartodbpositron",
    zoom_start=10.2).add_to(frame)
for lat, lon, name, address, population, pop_qin in zip(gp_prac_pop_df_1['Latitude'], 
gp_prac_pop_df_1['Longitude'], 
gp_prac_pop_df_1['Name'], 
gp_prac_pop_df_1['Full Address'], 
gp_prac_pop_df_1['Number of patients registered at GP practices in London'], 
gp_prac_pop_df_1['gp_pop_quintile']):
    folium.CircleMarker(
        [lat, lon],
        radius=0.06*(math.sqrt(population)),
        popup = folium.Popup('<b>' + 'Name: ' + '</b>'  + str(name) + '<br>'
                 '<b>' + 'Address: ' + '</b>' + str(address) + '<br>'
                 '<b>' + 'Number of Patients Registered: ' + '</b>' + str(population) + '<br>', max_width=len(address)*20),
        color='b',
        key_on = pop_qin,
        threshold_scale=[0,1,2,3,4,5],
        fill_color=colordict[pop_qin],
        fill=True,
        fill_opacity=0.8
        ).add_to(fig_2)
fig_2.save("assets/folium/folium_obj.html", "w")
fig_2